In [ ]:
# Install all the required Dependencies and Setup

!pip install labelme tensorflow tensorflow-gpu opencv-contrib-python matplotlib albumentations

In [ ]:
import os
import cv2
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Reshape, Dropout
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.models import load_model

In [ ]:
try:
    from google.colab import drive
    print('Running on Google colab...')
    drive.mount('/content/drive') 
except:
    print('Running on local machine...')

In [ ]:
# Load the images and the data

trainimages = tf.data.Dataset.list_files('/content/drive/MyDrive/aug_data/train/images/*.jpg',shuffle=False)
testimages = tf.data.Dataset.list_files('/content/drive/MyDrive/aug_data/test/images/*.jpg',shuffle=False)
valimages = tf.data.Dataset.list_files('/content/drive/MyDrive/aug_data/val/images/*.jpg',shuffle=False)

In [ ]:
# Load the images

def load_image(x): 
    byte_img = tf.io.read_file(x) # read the file as bytes
    img = tf.io.decode_jpeg(byte_img) # convert to jpeg
    return img

In [ ]:
trainimages = trainimages.map(load_image) # map is used to apply the image_load function to every image path obtained from tensor flow
trainimages = trainimages.map(lambda x: tf.image.resize(x, (250,250)))
trainimages = trainimages.map(lambda x: x/255)

testimages = testimages.map(load_image)
testimages = testimages.map(lambda x: tf.image.resize(x, (250,250)))
testimages = testimages.map(lambda x: x/255)

valimages = valimages.map(load_image)
valimages = valimages.map(lambda x: tf.image.resize(x, (250,250)))
valimages = valimages.map(lambda x: x/255)

In [ ]:
# Preparing the Labels

def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
    return [label['keypoints']]

In [ ]:
trainlabels = tf.data.Dataset.list_files('/content/drive/MyDrive/aug_data/train/labels/*.json', shuffle=False)
trainlabels = trainlabels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

testlabels = tf.data.Dataset.list_files('/content/drive/MyDrive/aug_data/test/labels/*.json', shuffle=False)
testlabels = testlabels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

vallabels = tf.data.Dataset.list_files('/content/drive/MyDrive/aug_data/val/labels/*.json', shuffle=False)
vallabels = vallabels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

# Combining Labels and Images for Iris Detection

train = tf.data.Dataset.zip((trainimages, trainlabels))
train = train.batch(16)
train = train.prefetch(4)

test = tf.data.Dataset.zip((testimages, testlabels))
test = test.batch(16)
test = test.prefetch(4)

val = tf.data.Dataset.zip((valimages, vallabels))
val = val.batch(16)
val = val.prefetch(4)

# Viewing Samples

datasamples = train.as_numpy_iterator()

res = datasamples.next()

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx]
    sample_coords = res[1][0][idx]
    
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[:2], [250,250]).astype(int)), 2, (255,0,0), -1)
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[2:], [250,250]).astype(int)), 2, (0,255,0), -1)
    
    ax[idx].imshow(sample_image)

# Building the model by creating the neural network

model = Sequential([
    Input(shape=(250,250,3)), 
    ResNet152V2(include_top=False, input_shape=(250,250,3)),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(256, 3, 2, padding='same', activation='relu'),
    Conv2D(256, 2, 2, activation='relu'),
    Dropout(0.05),
    Conv2D(4, 2, 2),
    Reshape((4,))
])

# Setting up Losses and Optimizer

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001, decay=0.0007)
loss = tf.keras.losses.MeanSquaredError()

model.compile(optimizer, loss)

# Sense Check predictions

X, y = train.as_numpy_iterator().next()

scores = model.predict(X)

# Training the model for 100 epochs
hist = model.fit(train, epochs=10, validation_data=val)

# Viewing the Loss Plots

plt.plot(hist.history['loss'], color='blue', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val loss')
plt.suptitle('Loss')
plt.autoscale() 
plt.legend()
plt.show()

# Making the Predictions on Test

testdata = test.as_numpy_iterator()

testsample = testdata.next()

yhat = model.predict(testsample[0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = testsample[0][idx]
    sample_coords = yhat[idx]
    
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[:2], [250,250]).astype(int)), 2, (255,0,0), -1)
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[2:], [250,250]).astype(int)), 2, (0,255,0), -1)
    
    ax[idx].imshow(sample_image)

In [ ]:
# Viewing the Loss Plots

hist.history
plt.plot(hist.history['loss'], color='blue', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val loss')
plt.suptitle('Loss')
plt.autoscale() 
plt.legend()
plt.show()

In [ ]:
# Performing Real Time Detection

cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    
    frame = frame[50:500,50:500,:] 
    rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(rgb_img, (250,250))
    
    yhat = model.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[0,:4]
    
    cv2.circle(frame, tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)), 2, (255,0,0), -1)
    cv2.circle(frame, tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 2, (0,255,0), -1)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Saving the Model

model.save('eyetrackerresnet.h5')
model = load_model('eyetrackerresnet.h5')
model.predict(testsample[0])